In [1]:
import keras
import numpy as np
import tensorflow as tf
import cv2
import os

Using TensorFlow backend.


In [2]:
from keras.applications.resnet50 import ResNet50
from keras.applications.densenet import DenseNet121
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.nasnet import NASNetMobile
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

In [3]:
# Data Handler
trainList = []
trainScores = []
valList = []
valScores = []

realList = []

with open('train-scores.txt','r') as f:
    trainData = f.readlines()
    
for index in range(16800):
    line = trainData[index]
    space = line.find(' ')
    imgName = line[0:space]
    imgScore = float(line[space+1:-1])
    img = cv2.imread(os.path.join('synthetic',imgName))
    trainList.append(img)
    trainScores.append(imgScore)
print('train over')
trainList = np.array(trainList)
trainScores = np.array(trainScores)

with open('val-scores.txt','r') as f:
    valData = f.readlines()
    
for index in range(4200):
    line = valData[index]
    space = line.find(' ')
    imgName = line[0:space]
    imgScore = float(line[space+1:-1])
    img = cv2.imread(os.path.join('synthetic',imgName))
    valList.append(img)
    valScores.append(imgScore)
print('val over')
valList = np.array(valList)
valScores = np.array(valScores)

for index in range(3500):
    imgName = str(index+1)+'.JPG'
    img = cv2.imread(os.path.join('real',imgName))
    realList.append(cv2.resize(img,(224,224)))
print('real over')
realList = np.array(realList)

train over
val over


In [3]:
inputImg = Input(shape=(224,224,3))
# ResNet-50
resModel = ResNet50(include_top=False, input_shape=(224,224,3), pooling='avg')
featureRes = resModel(inputImg)
# DenseNet
# denseModel = DenseNet121(include_top=False, input_shape=(224,224,3), pooling='avg')
# featureDense = denseModel(inputImg)
# GoogleNet
# googleModel = InceptionV3(include_top=True, input_shape=(224,224,3), pooling='avg')
# featureGoogle = googleModel(inputImg)
# MobileNet
# mobileModel = MobileNetV2(include_top=True, input_shape=(224,224,3), pooling='avg')
# featureMobile = mobileModel(inputImg)
# NASNet
# nasModel = NASNetMobile(include_top=False, input_shape=(224,224,3), pooling='avg')
# featureNas = nasModel(inputImg)

output = Dense(1)(featureRes)

In [4]:
model = Model(inputImg, output)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
# nasModel.summary()

In [6]:
savedir = 'savemodel'
filepath = "nasModel_{epoch:02d}-{val_loss:.2f}.hdf5"
model_checkpoint = ModelCheckpoint(os.path.join(savedir,filepath), verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', verbose=1)

In [7]:
model.fit(x=trainList, y=trainScores, epochs=20, batch_size=16,\
                    validation_data = [valList, valScores],\
                    callbacks=[model_checkpoint])

Train on 16800 samples, validate on 4200 samples
Epoch 1/20
16800/16800 [==============================] - 655s 39ms/step - loss: 0.0413 - mean_absolute_error: 0.1456 - val_loss: 0.0204 - val_mean_absolute_error: 0.1084

Epoch 00001: val_loss improved from inf to 0.02040, saving model to savemodel\nasModel_01-0.02.hdf5
Epoch 2/20
16800/16800 [==============================] - 649s 39ms/step - loss: 0.0177 - mean_absolute_error: 0.1032 - val_loss: 0.0312 - val_mean_absolute_error: 0.1467

Epoch 00002: val_loss did not improve from 0.02040
Epoch 3/20
16800/16800 [==============================] - 648s 39ms/step - loss: 0.0164 - mean_absolute_error: 0.0997 - val_loss: 0.1122 - val_mean_absolute_error: 0.2517

Epoch 00003: val_loss did not improve from 0.02040
Epoch 4/20
16800/16800 [==============================] - 649s 39ms/step - loss: 0.0170 - mean_absolute_error: 0.1011 - val_loss: 0.1112 - val_mean_absolute_error: 0.2619

Epoch 00004: val_loss did not improve from 0.02040
Epoch 5/20

In [5]:
model.load_weights('savemodel/resModel_04-0.02.hdf5')

In [23]:
preScores = model.predict(realList)
np.savetxt('resResults.txt', preScores, fmt='%.4f', delimiter='\n')

In [23]:
img = cv2.imread('paper/IMG_2378.JPG')
img = cv2.resize(img,(224,224))
img = np.expand_dims(img, axis=0)
model.predict(img)

array([[ 0.5697006]], dtype=float32)